In [7]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import copy
from torch.utils.data import DataLoader,  TensorDataset
from sklearn.utils import shuffle
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchsummary import summary
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import data_challenge_library as dcl
from sklearn.preprocessing import LabelBinarizer
import utility_library as ulb

In [2]:
object_table = dcl.load_table(features = "Savic", have_images = True)

Original Table has 446487 sources
Keeping 432767 labeled sources
Keeping only the required 62 features
Keeping 142963 with all features available
Keeping 142963 with available cutouts
class
Star    56532
Gal     53331
Qso     32780
Agn       320
Name: count, dtype: int64


In [4]:
train, validation, test = dcl.prepare_sample(object_table)
train.get_images(dcl.load_images(train.objectID, as_tensor=True))
test.get_images(dcl.load_images(test.objectID, as_tensor = True))
validation.get_images(dcl.load_images(validation.objectID, as_tensor = True))

Returning train, validation and test objects


In [27]:
train.get_dataloader(batch_size = 40)
validation.get_dataloader(batch_size = 40)
test.get_dataloader(batch_size = 40)

Loading images in the dataset


In [28]:
class merged_model(nn.Module):
    def __init__(self, process_tabular, process_image, process_flattened_image, process_both):
        super(merged_model, self).__init__()

        self.process_tabular = process_tabular
        self.process_image = process_image
        self.process_flattened_image = process_flattened_image
        self.process_both = process_both

    def forward(self, tabular, image):
        image = self.process_image(image)
        image = torch.flatten(image, start_dim = 1, end_dim = -1)
        image = self.process_flattened_image(image)    
        tabular = self.process_tabular(tabular)
        
        out = torch.concat([image, tabular], dim =1)
        out = self.process_both(out)    
        return out

In [80]:
class merged_model_explicit(nn.Module):
    def __init__(self):
        super(merged_model_explicit, self).__init__()
        
        self.process_tabular = nn.Sequential(nn.Linear(62, 62, bias = True), 
                                             nn.LeakyReLU(), nn.Linear(62,30, bias=True),
                                             nn.LeakyReLU())

        self.process_image = nn.Sequential(nn.MaxPool2d(kernel_size = 2, stride = 1),
                                           nn.LeakyReLU(),
                                           nn.Conv2d(3, 1, kernel_size=1),  nn.LeakyReLU())
        self.process_flattened_image = nn.Sequential(nn.Linear(225, 30), nn.LeakyReLU(),
                                                     nn.Linear(30, 30), nn.LeakyReLU())
        
        self.process_both = nn.Sequential(nn.Linear(60, 30), nn.LeakyReLU(), 
                                          nn.Linear(30, 3), nn.LeakyReLU() )
     

    def forward(self, tabular, image):
        image = self.process_image(image)
        print(image.shape)
        image = torch.flatten(image, start_dim = 1, end_dim = -1)
        image = self.process_flattened_image(image)    
        tabular = self.process_tabular(tabular)
        
        out = torch.concat([image, tabular], dim =1)
        out = self.process_both(out)    
        return out

In [81]:
process_tabular = ulb.general_dense(input_size = 62, output_size = 30,  hidden_sizes =[62],
                  activation_function = nn.LeakyReLU)
process_image = ulb.general_convo2d(out_channels = 1, conv_kernel = 1, conv_stride = 1, conv_padding = 0,
                             pool_kernel = 2)
process_flattened_image = ulb.general_dense(input_size = process_image.Nout_tot, output_size = 30, 
                                                    hidden_sizes =[30])
process_both = ulb.general_dense(input_size = 60, output_size = 3, hidden_sizes =[30])

combined_model = merged_model(process_tabular, process_image, process_flattened_image, process_both)

In [55]:
def train_routine(dataloader, model, loss_fn, optimizer, verbose = True):
    losses = []
    num_batches = len(dataloader)
    for batch, (features, images,  labels) in enumerate(dataloader): 
        output = model(features, images)
        loss = loss_fn(output, labels)

        # Backpropagation
        optimizer.zero_grad()    # Clear the gradient
        loss.backward()          # Compute the gradient (??)
        optimizer.step()         # update model weights

        
        if batch == round(num_batches/2):
            losses.append(loss.item())
            if verbose:
                print(f"loss: {loss:>7f}")

    return losses

def test_routine(dataloader, model, loss_fn, verbose = True):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0
    model.eval() 
    with torch.no_grad():
        for features, images, labels in dataloader:
            output = model(features, images)
            test_loss += loss_fn(output, labels).item()

    test_loss /= num_batches
    if verbose:
        print(f" Avg test loss      : {test_loss:>8f}")

    return test_loss

In [31]:
EPOCHS = 20
SEED = 123
loss_fn = torch.nn.CrossEntropyLoss()  #torch.nn.CrossEntropyLoss
optimizer = torch.optim.SGD(combined_model.parameters(), lr=0.001, weight_decay=0.001)

In [32]:
train_loss = []
test_loss  = []
for t in range(EPOCHS):
    print(f"Epoch {t+1}---------------------")
    train_loss.append(train_routine(train.dataloader, combined_model, loss_fn, optimizer, verbose = False))
    test_loss.append(test_routine(validation.dataloader, combined_model, loss_fn, verbose = True))
print("Done!")

Epoch 1---------------------
 Avg test loss      : 1.065625
Epoch 2---------------------
 Avg test loss      : 0.995638
Epoch 3---------------------
 Avg test loss      : 0.780467
Epoch 4---------------------
 Avg test loss      : 0.596399
Epoch 5---------------------
 Avg test loss      : 0.549283
Epoch 6---------------------
 Avg test loss      : 0.534706
Epoch 7---------------------
 Avg test loss      : 0.528098
Epoch 8---------------------
 Avg test loss      : 0.524048
Epoch 9---------------------
 Avg test loss      : 0.521516
Epoch 10---------------------
 Avg test loss      : 0.519525
Epoch 11---------------------
 Avg test loss      : 0.518073
Epoch 12---------------------
 Avg test loss      : 0.516745
Epoch 13---------------------
 Avg test loss      : 0.515742
Epoch 14---------------------
 Avg test loss      : 0.514770
Epoch 15---------------------
 Avg test loss      : 0.513926
Epoch 16---------------------
 Avg test loss      : 0.513143
Epoch 17---------------------
 Av

In [82]:
model = merged_model_explicit()
batch_f, batch_i, batch_lab = next(iter(train.dataloader))
yhat = model(batch_f, batch_i)


torch.Size([40, 1, 15, 15])


In [85]:
from torchviz import make_dot

make_dot(yhat,params = dict(model.named_parameters())).render("rnn_torchviz",format="png")

'rnn_torchviz.png'

In [47]:
list(combined_model.named_parameters())

[('process_tabular.full_sequence.0.weight',
  Parameter containing:
  tensor([[ 0.0971,  0.0882,  0.1191,  ...,  0.0964,  0.1005, -0.0140],
          [ 0.0443,  0.1372, -0.0474,  ..., -0.0995, -0.1008,  0.0275],
          [ 0.0921,  0.0330,  0.1203,  ..., -0.0782, -0.0305, -0.0047],
          ...,
          [-0.0810, -0.0962,  0.1133,  ...,  0.0230, -0.0425, -0.0936],
          [ 0.0850,  0.0824, -0.0698,  ...,  0.0069, -0.1047,  0.0090],
          [ 0.1174,  0.0236, -0.0436,  ..., -0.0360,  0.0243, -0.0787]],
         requires_grad=True)),
 ('process_tabular.full_sequence.0.bias',
  Parameter containing:
  tensor([ 0.0725,  0.1362,  0.0226, -0.0930,  0.0832,  0.0777, -0.1061,  0.1225,
           0.0129, -0.0747, -0.0088,  0.0158, -0.0518,  0.0497,  0.0917, -0.0481,
           0.1120, -0.0863,  0.1171, -0.0354,  0.0878, -0.0188, -0.0674, -0.0578,
           0.0466,  0.0287, -0.0098, -0.0690,  0.1169, -0.0541, -0.0655, -0.0546,
           0.0833,  0.1349,  0.0332,  0.0433,  0.1252,  0.1

In [51]:
process_image.Nout_pool

15.0

In [52]:
process_image.Nout_tot

225

In [53]:
15*15

225

In [71]:
y = torch.rand(40,16,16)

In [72]:
x = nn.Sequential(nn.MaxPool2d(kernel_size = 2, stride = 1),
                                           nn.LeakyReLU())

In [73]:
x(y).shape

torch.Size([40, 15, 15])

In [74]:
a = torch.flatten(x(y), start_dim = 1, end_dim = -1)

In [75]:
a.shape

torch.Size([40, 225])

In [66]:
a.flatten().shape

torch.Size([225])